Complete codes to processing CoastCams

# 📖 Table of Contents
- [Get camera intrinsics and extrinsics](#Get-camera-intrinsics-and-extrinsics)
- [Define timestacks for ARGUS camera](#Define-timestacks-for-ARGUS-camera)
- [Rectify image products](#Rectify-image-products)
    - [Example: Plot image rectification from netcdf](##Example-plot-of-rectified-image)
    - [Example: Rectify single image](##Rectifying-single-image)
- [Extract runup](#Extract-runup)
    - [Example: Runup from single image](##Runup-from-single-image)
    - [Codes for DEM plots](##DEM-plots)
- [Extract shorelines](#Extract-shorelines)
    - [Example: Shoreline from single image](##Shoreline-from-single-image)


## Get camera intrinsics and extrinsics

## Define timestacks for ARGUS camera

In [ ]:
import sys
import os
import json
if 'CODES' not in sys.path:
    sys.path.append('CODES')
    
from ImageHandler import ImageHandler
import utils_runup 

configPath = "config.json"
sample_img_path = "DATA/DATA/images/1737649800.Thu.Jan.23_16_30_00.GMT.2025.CACO03.c1.snap.jpg"

#yaml_filename = "filename dictionary of coordinates E,N,Z,U,V - end in .yaml"
yaml_filename = 'eyample.yaml'
#pix_filename = "filename for .pix file - end in .pix"
pix_filename = 'eyample.pix'

img_handle = ImageHandler(imagePath = sample_img_path, configPath = configPath)

img_handle.read_metadata()
metadata = img_handle.metadata
config = img_handle.config

productsPath = config.get("productsPath", {})
pixsaveDir = config.get("piysaveDir", {})

utils_runup.define_UV_transects(productsPath, 
                    metadata['intrinsics'], 
                    metadata['eytrinsics'], 
                    pixsaveDir=pixsaveDir, 
                    yaml_filename = yaml_filename, 
                    pix_filename = pix_filename)         



## Rectify image products

In [ ]:
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageDatastore

# Set up datastore for all files in imageDir
datastore = ImageDatastore(configPath = 'config.json')
datastore.image_stats()
# Only keep image products
datastore.keep_images_by_type(['snap', 'timex', 'var', 'bright', 'dark'])
datastore.image_stats()

# Load all images to datastore
#datastore.initialize_image_handlers()

# Rectify each image onto grid provided in productsPath
datastore.apply_to_all_images('rectify_image')

# Save rectified image to netcdf
datastore.apply_to_all_images('save_to_netcdf')

# Create video of brightest oblique images - can also add start_time and end_time
datastore.create_video(image_type = 'bright', frame_rate = 12, camera = '2')#, end_time=datetime(2024, 8,17,00,00,00, tzinfo=timezone.utc))

# Rectify images, merge camers and save full image with text overlap of site, camera, time. Does not store rectified/merged images.
#datastore.merge_images_fast()


### Example plot of rectified image

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
ds = xr.open_dataset("DATA/DATA/results/netcdf/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.timex.nc")
ds
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle(f"Rectified Image - {ds.attrs['name']}", fontsize=14)

# First subplot: pcolor
ax1 = axes[0]
pcolor_plot = ax1.pcolor(ds['Eastings'], ds['Northings'], ds['Ir'], shading='auto')
ax1.scatter(ds.attrs['origin_easting'], ds.attrs['origin_northing'], c='r')
ax1.set_xlabel("Eastings")
ax1.set_ylabel("Northings")

# Second subplot: imshow
ax2 = axes[1]
im = ax2.imshow(
    ds['Ir'],
    extent=[ds['localX'][0, 0], ds['localX'][-1, -1], ds['utmY'][0, 0], ds['utmY'][-1, -1]],
    origin='lower'
)
ax2.set_xlabel("utm X")
ax2.set_ylabel("Local Y")

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for title


### Rectifying single image

In [ ]:

import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageHandler

imagePath = 'path/to/image'
image_handler = ImageHandler(imagePath = imagePath, configPath = 'config.json')
image_handler.rectify_image()
image_handler.save_to_netcdf()

## Extract runup

In [ ]:
import sys
import json
if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageDatastore

with open('config.json', "r") as f:
    config = json.load(f)

# Split .tiff files from i2RGUS into individual transects
if config.get('split_tiff', False) is True:
    import utils_runup
    utils_runup.split_tiff(config)

# Set up datastore for all files in imageDir
datastore = ImageDatastore(configPath = config)
# Remove image products -> keep transects
datastore.remove_images_by_type(['snap', 'timex', 'var', 'bright', 'dark'])
datastore.image_stats()

# Get runup from segformer model (segformerCodeDir/segformerModel), rectify onto DEM (demPath) and save to netcdf
datastore.get_runup_from_timestacks(segformer_flag=True, save_flag=True)


### Runup from single image

In [ ]:
import sys
import json
if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageHandler
with open('config.json', "r") as f:
    config = json.load(f)

# Set up image
img_handle = ImageHandler(imagePath="/mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/images/runup_play/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.ras_transect6.png", configPath = config)
# Run segformer model
img_handle.run_segformer_on_timestack()
# Extract runup
img_handle.get_runup_from_segformer()
# Project horizontal runup to vertical and get runup stats
img_handle.compute_runup()
# Save to netcdf
img_handle.save_to_netcdf()

Running Segformer for 1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.ras_transect6
/mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_runup_example/CODES/
Saved overlay image: /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/results/runup/overlay_1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.ras_transect6.jpg
Getting runup
getting tide
computing runup stats
Saving runup


/mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/Shorescan/ShoreScan_clean_example/CODES/ImageHandler.py:1026: UserWarning: no explicit representation of timezones available for np.datetime64
  T = np.array([self.datetime + timedelta(seconds = t) for t in t_sec], dtype = np.datetime64)


All data saved to /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/results/netcdf/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.ras_transect6.nc


### DEM plots

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

# Plot the surface
fig.add_trace(go.Surface(z=z[::100], x=X[::100], y=Y[::100], colorscale='earth', opacity=1, showscale=False,
    colorbar=dict(title="Elevation (m)", len=0.75, thickness=20),
    contours=dict(
        x=dict(show=True, color="black"),
        y=dict(show=True, color="black"),
        z=dict(show=True, highlightcolor="black", project_z=True)
    )))

# Scatter plot for extrinsics points (Ray origin)
fig.add_trace(go.Scatter3d(x=[extrinsics['x']], y=[extrinsics['y']], z=[extrinsics['z']], mode='markers', marker=dict(size=5, color='blue', opacity=0.7), name='Extrinsics'))

transect_data = coords_grid['transect_0']
for idx, Points_w_set in enumerate(transect_data['xyz']):
    
    fig.add_trace(go.Scatter3d(
            x=[Points_w_set[0]], 
            y=[Points_w_set[1]], 
            z=[Points_w_set[2]], 
            mode='markers', 
            marker=dict(size=5, opacity=0.7, color='black'),
            name=f'Crossing point'  # Naming each set based on the iteration
        ))
fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=3, y=1, z=1),
        xaxis=dict(
            title='X-axis',
            range=[extrinsics['x']-100, extrinsics['x']+150]  # Replace min_x and max_x with your desired limits
        ),
        yaxis=dict(
            title='Y-axis',
            range=[extrinsics['y']-150, extrinsics['y']+150]  # Replace min_y and max_y with your desired limits
        ),
        zaxis=dict(
            title='Z-axis',
            range=[-2,22]  # Replace min_x and max_x with your desired limits
        ),
    ),
    title=f'3D Surface at {45} degrees',  # Change 45 to your angle_deg value
    margin=dict(r=10, b=10, l=10, t=40) # Adjust margins as needed
)
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Plot the surface
fig.add_trace(go.Surface(z=z[::100], x=X[::100], y=Y[::100], colorscale='earth', opacity=1, showscale=False,
    colorbar=dict(title="Elevation (m)", len=0.75, thickness=20),
    contours=dict(
        x=dict(show=True, color="black"),
        y=dict(show=True, color="black"),
        z=dict(show=True, highlightcolor="black", project_z=True)
    )))

fig.add_trace(go.Scatter3d(
        x=grid['eastings_shoreline'], 
        y=grid['northings_shoreline'], 
        z=grid['elevation_shoreline'], 
        mode='markers', 
        marker=dict(size=5, opacity=0.7, color = 'black'),
        name=f'Shoreline'  # Naming each set based on the iteration
    ))

fig.add_trace(go.Scatter3d(x=t2['Hrunup_utm'].values[:,0], y=t2['Hrunup_utm'].values[:,1], z=t2['TWL'].values, mode='markers', marker=dict(size=5, opacity=0.7),
        name=f'Transect 2'))
fig.add_trace(go.Scatter3d(x=t3['Hrunup_utm'].values[:,0], y=t3['Hrunup_utm'].values[:,1], z=t3['TWL'].values, mode='markers', marker=dict(size=5, opacity=0.7),
        name=f'Transect 3'))
fig.add_trace(go.Scatter3d(x=t4['Hrunup_utm'].values[:,0], y=t4['Hrunup_utm'].values[:,1], z=t4['TWL'].values, mode='markers', marker=dict(size=5, opacity=0.7),
        name=f'Transect 4'))
fig.add_trace(go.Scatter3d(x=t5['Hrunup_utm'].values[:,0], y=t5['Hrunup_utm'].values[:,1], z=t5['TWL'].values, mode='markers', marker=dict(size=5, opacity=0.7),
        name=f'Transect 5'))
fig.add_trace(go.Scatter3d(x=t6['Hrunup_utm'].values[:,0], y=t6['Hrunup_utm'].values[:,1], z=t6['TWL'].values, mode='markers', marker=dict(size=5, opacity=0.7),
        name=f'Transect 6'))


# Labels and title
fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=3, y=1, z=1),
        xaxis=dict(
            title='X-axis',
            range=[grid.attrs['origin_easting'], grid.attrs['origin_easting']+100]  # Replace min_x and max_x with your desired limits
        ),
        yaxis=dict(
            title='Y-axis',
            range=[grid.attrs['origin_northing']-50, grid.attrs['origin_northing']+80]  # Replace min_y and max_y with your desired limits
        ),
        zaxis=dict(
            title='Z-axis',
            range=[-2,6]  # Replace min_x and max_x with your desired limits
        ),
    ),
    title=f'3D Surface at {45} degrees',  # Change 45 to your angle_deg value
    margin=dict(r=10, b=10, l=10, t=40) # Adjust margins as needed
)


# Show plot
fig.show()


## Extract shorelines

In [ ]:
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageDatastore

# Set up datastore for all files in imageDir
datastore = ImageDatastore(configPath = 'config.json')
# Only run on bright images
datastore.keep_images_by_type([ 'bright'])
datastore.image_stats()

# Rectify each image onto grid provided in productsPath
datastore.apply_to_all_images('rectify_image')
# Extract shoreline from oblique image and rectify to DEM (demPath) - will processes matching bright and timex
datastore.process_shorelines(make_plots=False, save_flag = True)

### Shoreline from single image

In [ ]:
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')
from ImageHandler import ImageHandler

# Set up image
img_handle = ImageHandler(configPath = 'config.json', imagePath = '/mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/images/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.bright.jpg')
# Rectify image
img_handle.rectify_image(dem_flag = True)
# Extract brightest max shoreline
img_handle.process_bright()
# Project shoreline onto DEM (demPath)
img_handle.rectify_shoreline()
# Save shoreline to netcdf
img_handle.save_to_netcdf()